In [53]:
# Import required libraries
# !pip install -q  gradio

import os
import numpy as np
import pandas as pd
import face_recognition
from PIL import Image, ImageDraw
from fastai.vision.all import *
from fastai.vision.widgets import *
from pathlib import Path
import matplotlib.pyplot as plt
import random
import gradio as gr
from rembg import remove
from PIL import Image


In [4]:
model_path = Path('./42amman_face_model.pkl')
features_path = Path('./42amman_face_features.npy')
profiles_path = Path('./amman_profile_images')

# Verify files exist
print(f"Model exists: {model_path.exists()}")
print(f"Feature vectors exist: {features_path.exists()}")
print(f"Profile images exist: {profiles_path.exists()}")

Model exists: True
Feature vectors exist: True
Profile images exist: True


In [25]:
# Load the model
learn = load_learner(model_path)
print("Model loaded successfully!")

# Load feature vectors
feature_vectors = np.load(features_path, allow_pickle=True).item()
print(f"Loaded feature vectors for {len(feature_vectors)} students")

Model loaded successfully!
Loaded feature vectors for 767 students


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")


In [ ]:
def process_image(img_path, output_path):
    """Detect face in image, crop, and save to output path"""
    try:
        # Load image using face_recognition
        if isinstance(img_path, str) or isinstance(img_path, Path):
            image = face_recognition.load_image_file(img_path)
        else:  # Assume it's a numpy array
            image = img_path

        # Find all face locations in the image
        face_locations = face_recognition.face_locations(image, model="cnn")

        if not face_locations:
            print(f"No face found in the image")
            return False

        # For simplicity, we'll use the first face found
        top, right, bottom, left = face_locations[0]

        # Add some margin to the face crop (20% of face size)
        height, width = bottom - top, right - left
        margin_h, margin_w = int(height * 0.2), int(width * 0.2)

        # Adjust boundaries with margins and ensure they're within image bounds
        img_h, img_w = image.shape[:2]
        top = max(0, top - margin_h)
        bottom = min(img_h, bottom + margin_h)
        left = max(0, left - margin_w)
        right = min(img_w, right + margin_w)

        # Crop the image to focus on the face
        face_image = image[top:bottom, left:right]
        pil_image = Image.fromarray(face_image)

        # Resize to a standard size (224x224 for resnet34)
        pil_image = pil_image.resize((224, 224), Image.Resampling.LANCZOS)

        # Save the processed image
        pil_image.save(output_path)
        return True
    except Exception as e:
        print(f"Error processing image: {e}")
        return False

def extract_features(learn, img_path):
    """Extract features from the penultimate layer of the model"""
    import torch

    # Load and transform the image
    img = PILImage.create(img_path)

    # Create a test batch with a single image
    batch = learn.dls.test_dl([img])

    # Determine the device and move data appropriately
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    print(f"Using device: {device}")

    # Get the model's feature extractor (everything except the final layer)
    feature_extractor = learn.model[:-1].to(device)

    # Ensure the model is in eval mode
    feature_extractor.eval()

    # Extract features with gradient calculation disabled for efficiency
    with torch.no_grad():
        # Get the first batch from the dataloader
        batch_data = first(batch)

        # Handle different batch data structures
        if isinstance(batch_data, tuple):
            # Always take the first element from the tuple, which should be the input tensor
            x = batch_data[0]
        else:
            # It's already a tensor
            x = batch_data

        # Move tensor to device
        x = x.to(device)

        # Get the embeddings/activations from the feature extractor
        activations = feature_extractor(x)

    # Convert to numpy array (first taking it back to CPU if needed)
    return activations[0].cpu().numpy()

def recognize_face(img_path, threshold=0.7):
    """Recognize a face in the given image

    Args:
        img_path: Path to the query image
        threshold: Cosine similarity threshold (higher = more strict)

    Returns:
        List of potential matches with similarity scores
    """
    # Create working directory if it doesn't exist
    work_dir = Path('./')
    work_dir.mkdir(exist_ok=True)

    # Process the query image to extract face
    query_face_path = work_dir/'query_face.jpg'
    face_found = process_image(img_path, query_face_path)

    if not face_found:
        return "No face detected in the query image", None

    # Extract features from the query face
    query_features = extract_features(learn, query_face_path)

    # Debug information
    print(f"Query feature shape: {query_features.shape}")

    # Calculate cosine similarity with all known faces
    similarities = {}
    for name, features in feature_vectors.items():
        # Ensure features are flattened to 1D if needed
        flat_query = query_features.flatten()
        flat_features = np.array(features).flatten()

        # Print shape information for first item to debug
        if len(similarities) == 0:
            print(f"Database feature shape: {np.array(features).shape}")
            print(f"Flattened shapes - Query: {flat_query.shape}, DB: {flat_features.shape}")

        # Calculate cosine similarity
        similarity = np.dot(flat_query, flat_features) / (
            np.linalg.norm(flat_query) * np.linalg.norm(flat_features))

        # Store as single float value
        similarities[name] = float(similarity)

    # Sort by similarity score (highest first)
    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

    # Filter by threshold
    matches = [match for match in sorted_similarities if match[1] >= threshold]

    if not matches:
        return "No matches found above the similarity threshold", sorted_similarities[:5]

    return matches, query_face_path

# Test on a profile image (should match perfectly)
test_img = list(profiles_path.glob('*.jpg'))[10]
print(f"Testing recognition with image: {test_img.name}")

matches, query_face_path = recognize_face(test_img, threshold=0.8)

if isinstance(matches, str):
    print(matches)
else:
    print(f"Found {len(matches)} potential matches:")
    for name, score in matches[:5]:  # Show top 5 matches
        print(f"  - {name}: {score:.4f}")

def visualize_matches(query_face_path, matches, top_n=5):
    """Visualize the query face and top matches"""
    if isinstance(matches, str):
        plt.figure(figsize=(8, 6))
        plt.text(0.5, 0.5, matches, fontsize=14, ha='center')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        return

    # Get top N matches
    top_matches = matches[:min(top_n, len(matches))]

    # Create figure with query face and top matches
    fig = plt.figure(figsize=(15, 8))

    # Display query face
    ax = fig.add_subplot(1, len(top_matches) + 1, 1)
    query_img = plt.imread(query_face_path)
    ax.imshow(query_img)
    ax.set_title("Query Face", fontsize=14)
    ax.axis('off')

    # Display top matches
    for i, (name, score) in enumerate(top_matches):
        # Get the original profile image for this match
        match_img_path = profiles_path/f"{name}.jpg"

        if not match_img_path.exists():
            continue

        ax = fig.add_subplot(1, len(top_matches) + 1, i + 2)
        match_img = plt.imread(match_img_path)
        ax.imshow(match_img)
        ax.set_title(f"{name}\nScore: {score:.4f}", fontsize=12)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# Visualize the test results
if not isinstance(matches, str) and query_face_path is not None:
    visualize_matches(query_face_path, matches)
    


Testing recognition with image: sata.jpg
Using device: mps
Query feature shape: (512, 7, 7)
Database feature shape: (512, 7, 7)
Flattened shapes - Query: (25088,), DB: (25088,)
No matches found above the similarity threshold


In [27]:
def recognize_face_gradioe(image, threshold=0.7):
    """Recognize face for Gradio interface"""
    if image is None:
        return "Please upload an image"

    # Save the uploaded image temporarily
    temp_path = './image.png'
    Image.fromarray(image).save(temp_path)

    # Recognize the face
    matches, query_face_path = recognize_face(temp_path, threshold=threshold)

    if isinstance(matches, str):
        return matches

    # Format the results
    result = ""
    if len(matches) > 0:
        result += f"Found {len(matches)} potential matches:\n\n"
        for i, (name, score) in enumerate(matches[:5]):  # Show top 5 matches
            result += f"{i+1}. {name}: Confidence {score:.2%}\n"
    else:
        result = "No matches found above the threshold."

    return result

# Create the Gradio interface
iface = gr.Interface(
    fn=recognize_face_gradioe,
    inputs=[
        gr.Image(type="numpy", label="Upload a face image"),
        gr.Slider(minimum=0.5, maximum=0.95, value=0.7, step=0.05, label="Similarity Threshold")
    ],
    outputs=gr.Textbox(label="Recognition Results"),
    title="42 Amman Face Recognition",
    description="Upload a photo to check if it matches a student from 42 Amman."
)

# Launch the interface
iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://24035e623a3936fd4f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [39]:
# Create processed data directory
PROCESSED_PATH = Path('./processed_faces')
PROCESSED_PATH.mkdir(exist_ok=True)

def process_image(img_path, output_path):
    """Detect face in image, crop, and save to output path"""
    try:
        # Load image using face_recognition
        image = face_recognition.load_image_file(img_path)

        # Find all face locations in the image
        face_locations = face_recognition.face_locations(image, model="hog")

        if not face_locations:
            print(f"No face found in {img_path}")
            return False

        # For simplicity, we'll use the first face found
        top, right, bottom, left = face_locations[0]

        # Add some margin to the face crop (20% of face size)
        height, width = bottom - top, right - left
        margin_h, margin_w = int(height * 0.2), int(width * 0.2)

        # Adjust boundaries with margins and ensure they're within image bounds
        img_h, img_w = image.shape[:2]
        top = max(0, top - margin_h)
        bottom = min(img_h, bottom + margin_h)
        left = max(0, left - margin_w)
        right = min(img_w, right + margin_w)

        # Crop the image to focus on the face
        face_image = image[top:bottom, left:right]
        pil_image = Image.fromarray(face_image)

        # Resize to a standard size (224x224 for resnet34)
        pil_image = pil_image.resize((224, 224), Image.Resampling.LANCZOS)

        # Save the processed image
        pil_image.save(output_path)
        return True
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return False
    
process_image("yaltayeh_slack.jpg", "test.jpg")

True

In [88]:
from torchvision import transforms
from PIL import Image
import numpy as np
import numpy as np
from PIL import Image, ImageFilter
        
def remove_bg(img_tensor, output_path, mean=0.5, std=0.5):
    """
    Denormalizes and saves a grayscale image tensor to a file.
    """
    img = img_tensor.clone().detach().cpu()

    img = img * std + mean  # Denormalize
    img = img.clamp(0, 1)
    img_np = img.squeeze().numpy()  # shape: [H, W]
    img_uint8 = (img_np * 255).astype(np.uint8)

    img = Image.fromarray(img_uint8, mode='L')

    face_only = remove(img)

    return face_only

def process_image(img_path, output_path):
    """
    Detects face in an image, crops it, converts to grayscale, normalizes,
    and saves the result to output_path.
    """
    try:
        # Load image using face_recognition
        image = face_recognition.load_image_file(img_path, "L")

        # Detect face
        face_locations = face_recognition.face_locations(image, model="hog")
        if not face_locations:
            print(f"No face found in {img_path}")
            return False

        # Use first face found
        top, right, bottom, left = face_locations[0]
        height, width = bottom - top, right - left
        margin_h, margin_w = int(height * 0.2), int(width * 0.2)
        img_h, img_w = image.shape[:2]
        top = max(0, top - margin_h)
        bottom = min(img_h, bottom + margin_h)
        left = max(0, left - margin_w)
        right = min(img_w, right + margin_w)

        # Crop and resize
        face_image = image[top:bottom, left:right]
        pil_image = Image.fromarray(face_image).resize((224, 224), Image.Resampling.LANCZOS)

        # Convert to grayscale

        # Normalize with torchvision transforms
        transform = transforms.Compose([
            transforms.ToTensor(),  # Converts to [1, H, W] and scales to [0,1]
            transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
        ])
        img_tensor = transform(pil_image)
        
        # Save image (denormalized for viewing)
        face_only = remove_bg(img_tensor, output_path)

        face_only.convert("L").save(output_path)
        print(f"Saved processed image to {output_path}")
        return True

    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return False

process_image("yaltayeh_slack.jpg", "test.jpg")


Saved processed image to test.jpg


True

In [ ]:
input_image = Image.open("test.jpg")
output_image =   remove(input_image)# Removes background
output_image.save("face_only.png")


False